# API Data Ingestion and SQL Analysis

In this exercise, you will fetch data from a public API, store it in an SQLite database, and analyze it using SQL.

## Objectives

1. Fetch data from a public API and parse the JSON response.
2. Store the data in an SQLite database.
3. Write SQL queries to analyze the stored data.

In [ ]:
import requests
import sqlite3
import json
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the API endpoint
api_url = "https://api.coindesk.com/v1/bpi/currentprice.json"

# Make a request to the API
response = requests.get(api_url)
data = response.json()

# Print the fetched data
print(json.dumps(data, indent=4))

## Step 1: Store the Data

Write a function to store the fetched data in an SQLite database. The database should have a table called `prices` with the following schema:
- `currency_code` (TEXT)
- `rate_float` (REAL)
- `updated_at` (TEXT)

In [ ]:
def store_data(data):
    conn = sqlite3.connect('crypto_prices.db')
    c = conn.cursor()
    
    # Create the prices table
    c.execute('''
        CREATE TABLE IF NOT EXISTS prices (
            currency_code TEXT,
            rate_float REAL,
            updated_at TEXT
        )
    ''')
    
    # Insert data into the table
    for currency, details in data['bpi'].items():
        c.execute('''
            INSERT INTO prices (currency_code, rate_float, updated_at) 
            VALUES (?, ?, ?)
        ''', (
            currency,
            details['rate_float'],
            data['time']['updatedISO']
        ))
    
    conn.commit()
    conn.close()
    logging.info('Data stored successfully.')

# Store the fetched data
store_data(data)

## Step 2: Analyze the Data

Write SQL queries to analyze the data. For example, you might want to find:
- The currency with the highest rate.
- The currency with the lowest rate.
- The average rate across all currencies.

In [ ]:
def query_data():
    conn = sqlite3.connect('crypto_prices.db')
    c = conn.cursor()
    
    # Find the currency with the highest rate
    c.execute('SELECT currency_code, rate_float FROM prices ORDER BY rate_float DESC LIMIT 1')
    highest = c.fetchone()
    print(f'Highest Rate: {highest[0]} at {highest[1]}')
    
    # Find the currency with the lowest rate
    c.execute('SELECT currency_code, rate_float FROM prices ORDER BY rate_float ASC LIMIT 1')
    lowest = c.fetchone()
    print(f'Lowest Rate: {lowest[0]} at {lowest[1]}')
    
    # Find the average rate
    c.execute('SELECT AVG(rate_float) FROM prices')
    average = c.fetchone()
    print(f'Average Rate: {average[0]}')
    
    conn.close()

# Run the analysis
query_data()